In [1]:
import pandas as pd
import zemberek as zp
from zemberek.normalization.turkish_sentence_normalizer import TurkishSentenceNormalizer
from tqdm import tqdm
import signal

In [47]:
dataframe = pd.read_excel('Türkçe Nefret Söylemi Veri Seti_1k.xlsx', sheet_name='1000 Tweet')

dataframe = dataframe.iloc[1:, [1, 2]]

In [48]:
dataframe

,Tweet,Etiket
1,"başkan erdoğan'dan suriyeli mülteci mesajı! ""y...",hiçbiri
2,"arslan bulut yazdı ""300 metrekarelik konaklar ...",nefret
3,güzel bir amaçla yola çıkmış ama insan askerin...,hiçbiri
4,kimin parası bu türk halkının parası peki bizi...,nefret
5,galatasaraylı taraftarlara suriyeli diyenler d...,nefret
...,...,...
995,bu sadece kadın erkek meselesi mi hayır iyi in...,hiçbiri
996,eşcinsel genin olmadığına dair bilimsel çalışm...,nefret
997,eşcinsel çiftin düğün fotoğrafları viral oldu ...,hiçbiri
998,kral şakir necati karakteri... türk dizilerind...,hiçbiri


In [3]:
morphology = zp.TurkishMorphology.create_with_defaults()
normalizer = TurkishSentenceNormalizer(morphology)

2023-04-27 00:11:23,115 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 5.41452169418335



In [56]:
resultDict = {'__tweet__': [], '__label__': []}

cancaled = False

def handler(signum, frame):
    global cancaled
    cancaled = True
signal.signal(signal.SIGINT, handler)

for index, row in tqdm(dataframe.iterrows()):
    if cancaled:
        break
    try:
        normalizedSentence = normalizer.normalize(row[0])
        analysis = morphology.analyze_sentence(normalizedSentence)
        after = morphology.disambiguate(normalizedSentence, analysis)
    except:
        print("Error: ", normalizedSentence)
        continue
    words = {}
    for sentence in after.best_analysis():
        if (words.get(sentence.get_stem())):
            words[sentence.get_stem()] += 1
        else:
            words[sentence.get_stem()] = 1
    
    columns = set(list(words.keys()) + list(resultDict.keys())[2:])
    for column in columns:
        if column not in resultDict.keys():
            resultDict[column] = [0] * len(resultDict['__tweet__'])
        if column in words:
            resultDict[column].append(int(words[column]))
        else:
            resultDict[column].append(int(0))
    resultDict['__tweet__'].append(str(row[0]).replace('\n', ' '))
    resultDict['__label__'].append(str(row[1]) if row[1] != None and type(row[1]) is str else "")

999it [04:44,  3.51it/s]


In [50]:

list(resultDict.items())[1]


('__label__',
 ['hiçbiri',
  'nefret',
  'hiçbiri',
  'nefret',
  'nefret',
  'nefret',
  'hiçbiri',
  'hiçbiri',
  'nefret',
  'nefret',
  'nefret',
  'nefret',
  'nefret',
  'hiçbiri',
  'nefret',
  'nefret',
  'hiçbiri',
  'hiçbiri',
  'hiçbiri',
  'nefret',
  'hiçbiri',
  'nefret',
  'hiçbiri',
  'hiçbiri',
  'hiçbiri',
  'hiçbiri',
  'nefret',
  'hiçbiri',
  'hiçbiri',
  'nefret',
  'hiçbiri',
  'hiçbiri',
  'nefret',
  'hiçbiri',
  'hiçbiri',
  'hiçbiri',
  'hiçbiri',
  'hiçbiri',
  'nefret',
  'hiçbiri',
  'nefret',
  'hiçbiri',
  'hiçbiri',
  'nefret',
  'saldırgan',
  'hiçbiri',
  'nefret',
  'hiçbiri',
  'hiçbiri',
  'nefret',
  'hiçbiri',
  'nefret',
  'hiçbiri',
  'hiçbiri',
  'hiçbiri',
  'nefret',
  'nefret',
  'hiçbiri',
  'hiçbiri',
  'hiçbiri',
  'hiçbiri',
  'nefret',
  'hiçbiri',
  'hiçbiri',
  'nefret',
  'hiçbiri',
  'hiçbiri',
  'nefret',
  'hiçbiri',
  'hiçbiri',
  'hiçbiri',
  'hiçbiri',
  'nefret',
  'nefret',
  'nefret',
  'hiçbiri',
  'hiçbiri',
  'hiçbiri',


In [57]:
resultDataFrame = pd.DataFrame(resultDict)

In [58]:
resultDataFrame

,__tweet__,__label__,ver,mülteci,mesaj,"""",!,destek,başkan,aç,...,güya,düşkü,bilimsel,#güzel1şeyyaz,düğün,#eylül,necati,komik,fışkır,sök
0,"başkan erdoğan'dan suriyeli mülteci mesajı! ""y...",hiçbiri,1,1,1,2,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,"arslan bulut yazdı ""300 metrekarelik konaklar ...",nefret,1,0,0,2,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,güzel bir amaçla yola çıkmış ama insan askerin...,hiçbiri,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,kimin parası bu türk halkının parası peki bizi...,nefret,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,galatasaraylı taraftarlara suriyeli diyenler d...,nefret,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,bu sadece kadın erkek meselesi mi hayır iyi in...,hiçbiri,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
995,eşcinsel genin olmadığına dair bilimsel çalışm...,nefret,0,0,0,0,3,0,0,0,...,0,0,1,0,0,0,0,0,0,0
996,eşcinsel çiftin düğün fotoğrafları viral oldu ...,hiçbiri,0,0,0,0,0,0,0,0,...,0,0,0,1,1,1,0,0,0,0
997,kral şakir necati karakteri... türk dizilerind...,hiçbiri,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,2,0,0


In [59]:
resultDataFrame.to_csv("result.csv")